In [30]:
import os
import numpy as np
import torch

In [31]:
VAL_SCAN_NAMES = [line.rstrip() for line in open('/home/dhanalaxmi.gaddam/3D_New/scannet/meta_data/scannetv2_val.txt')] 
OBJ_CLASS_IDS = np.array([3,4,5,6,7,8,9,10,11,12,14,16,24,28,33,34,36,39])
GT_PATH = '/home/dhanalaxmi.gaddam/3D_New/scannet/scannet_train_detection_data' # path of data dumped with scripts in scannet folder 
PRED_PATH = '/home/dhanalaxmi.gaddam/3D_New/MRCH3D/11_May/result' # path of predictions 

In [32]:
def select_bbox(bboxes):
    choose_ids = []
    for i in range(bboxes.shape[0]):
        if bboxes[i,-1] in OBJ_CLASS_IDS:
            choose_ids.append(i)
    bboxes = bboxes[choose_ids]
    return bboxes

In [33]:

def mrmse(non_zero,count_pred, count_gt):
    ## compute mrmse
    nzero_mask=torch.ones(len(count_gt))
    if non_zero==1:
        nzero_mask=torch.zeros(len(count_gt))
        nzero_mask[count_gt!=0]=1
    mrmse=torch.pow(count_pred - count_gt, 2)
    mrmse = torch.mul(mrmse, nzero_mask)
    mrmse = torch.sum(mrmse, 0)
    nzero = torch.sum(nzero_mask, 0)
    mrmse = torch.div(mrmse, nzero)
    mrmse = torch.sqrt(mrmse)
#     print(mrmse.size())
    mrmse = torch.mean(mrmse)
    return mrmse

def rel_mrmse(non_zero,count_pred, count_gt):
    ## compute relative mrmse
    nzero_mask=torch.ones(len(count_gt))
    if non_zero==1:
        nzero_mask=torch.zeros(len(count_gt))
        nzero_mask[count_gt!=0]=1
    num = torch.pow(count_pred - count_gt, 2)
    denom = count_gt.clone()
    denom = denom+1
    rel_mrmse = torch.div(num, denom)
    rel_mrmse = torch.mul(rel_mrmse, nzero_mask)
    rel_mrmse = torch.sum(rel_mrmse, 0)
    nzero = torch.sum(nzero_mask, 0)
    rel_mrmse = torch.div(rel_mrmse, nzero)
    rel_mrmse = torch.sqrt(rel_mrmse)
    rel_mrmse = torch.mean(rel_mrmse)
    return rel_mrmse

In [34]:
def export_one_scan(scan_name):
    gt_bbox = np.load(os.path.join(GT_PATH, scan_name+'_all_noangle_40cls.npy'))
    gt_bbox = select_bbox(np.unique(gt_bbox,axis=0))
    pred_proposals = np.load(os.path.join(PRED_PATH, 'opt'+scan_name+'_nms.npy'))
    
    gt_count = []
    pred_count = []

    for i in range(gt_bbox.shape[0]):
        x = int(gt_bbox[i,-1])
        gt_count.append(x)
        

    for i in range(pred_proposals.shape[0]):
        x = int(OBJ_CLASS_IDS[int(pred_proposals[i,-1])-1])
        pred_count.append(x)
    
    return gt_count, pred_count

In [35]:

N = len(VAL_SCAN_NAMES)
cat = len(OBJ_CLASS_IDS)
c_ik = np.zeros((cat,N))
c_ik_hat_h3d = np.zeros((cat,N))
c = 0
p = 0
for ind, k in enumerate(OBJ_CLASS_IDS):

    for i, scan_name in enumerate(sorted(VAL_SCAN_NAMES)):
        gt_objects, h3d_pred_objects = export_one_scan(scan_name)
        c = gt_objects.count(k)
        p = h3d_pred_objects.count(k)
        c_ik[ind,i] = c
        c_ik_hat_h3d[ind,i] = p
print(c_ik)
print(c_ik_hat_h3d)



[[ 3.  3.  0. ...  0.  3.  4.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [11. 10. 22. ...  0.  0.  0.]
 ...
 [ 1.  1.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  1.  0.  0.]
 [ 2.  3.  0. ...  1.  5.  5.]]
[[ 1.  4.  0. ...  0.  3.  2.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [10. 10. 21. ...  0.  0.  1.]
 ...
 [ 0.  1.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  1.  0.  0.]
 [ 1.  2.  0. ...  1.  2.  5.]]


In [36]:
gc = np.sum(c_ik, axis=0)
pred_c = np.sum(c_ik_hat_h3d, axis = 0)

Class Agnostic counting : RMSE and Rel_RMSE

In [37]:
rmse_1 = mrmse(1, torch.from_numpy(pred_c).float(), torch.from_numpy(gc).float())
rel_rmse_1 = rel_mrmse(1, torch.from_numpy(pred_c).float(), torch.from_numpy(gc).float())
rmse_1, rel_rmse_1

(tensor(4.6547), tensor(1.0464))

Class specific counting : RMSE and Rel_RMSE

In [38]:
for i in range(c_ik.shape[0]):
    g_cat_1 = c_ik[:,i]
    p_cat_1 = c_ik_hat_h3d[:,i]
    rmse_cat0 = mrmse(0, torch.from_numpy(p_cat_1).float(), torch.from_numpy(g_cat_1).float())
    rmse_cat1 = mrmse(1, torch.from_numpy(p_cat_1).float(), torch.from_numpy(g_cat_1).float())
    rel_rmse_cat0 = rel_mrmse(0, torch.from_numpy(p_cat_1).float(), torch.from_numpy(g_cat_1).float())
    rel_rmse_cat1 = rel_mrmse(1, torch.from_numpy(p_cat_1).float(), torch.from_numpy(g_cat_1).float())
    print(rmse_cat0, rmse_cat1, rel_rmse_cat0, rel_rmse_cat1)

tensor(0.8498) tensor(1.2019) tensor(0.4665) tensor(0.6597)
tensor(0.6236) tensor(0.8819) tensor(0.3600) tensor(0.5092)
tensor(0.2357) tensor(0.5774) tensor(0.0491) tensor(0.1204)
tensor(1.9720) tensor(4.0620) tensor(0.7693) tensor(1.2895)
tensor(2.1985) tensor(4.5552) tensor(0.8232) tensor(1.4315)
tensor(0.4714) tensor(0.5345) tensor(0.3788) tensor(0.2887)
tensor(0.5774) tensor(0.8452) tensor(0.3932) tensor(0.5047)
tensor(0.5270) tensor(0.7071) tensor(0.3991) tensor(0.3801)
tensor(1.1547) tensor(2.) tensor(0.5401) tensor(0.9354)
tensor(1.5456) tensor(2.6458) tensor(0.6832) tensor(1.1107)
tensor(1.5635) tensor(2.7080) tensor(0.6545) tensor(1.1336)
tensor(0.5774) tensor(0.6455) tensor(0.4139) tensor(0.4167)
tensor(0.5270) tensor(0.6325) tensor(0.3727) tensor(0.3873)
tensor(0.6236) tensor(0.7746) tensor(0.4357) tensor(0.4916)
tensor(0.9428) tensor(1.2649) tensor(0.7164) tensor(0.4976)
tensor(1.5811) tensor(2.7568) tensor(0.8408) tensor(1.0702)
tensor(0.8165) tensor(1.2536) tensor(0.4073)